# two steps
after importing the combined_table_conflicts.csv
1. create the columns for every conflict type and for every chanel
2. separate the conflict columns in individual tables with non empty conflict column for every conflict type in every conflict table find the worst case for every mission name and produce a reduced table with only one mission per column

In [10]:
import pandas as pd
import numpy as np
import os
from IPython.display import display
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from ITUtils import *
from tqdm.notebook import tqdm


## Step 1
expand the conflicts table

In [11]:
# load the dataframe
filename = './databases/combined_tables_conflicts.csv'
df = pd.read_csv(filename, low_memory=False)
# load tpa dataframe
tpafile = './databases/TPAtable.csv'
tpadf = pd.read_csv(tpafile, low_memory=False)


In [12]:
print(df.columns)

Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.long_nom',
       ' com_el.prov', ' com_el.d_rcv', ' com_el.st_cur', ' orbit.orb_id',
       ' orbit.nbr_sat_pl', ' orbit.apog_km', ' orbit.perig_km',
       ' orbit.op_ht_km', ' s_beam.emi_rcp', ' s_beam.beam_name',
       ' grp.grp_id', ' grp.freq_min', ' grp.freq_max', ' grp.bdwdth',
       ' grp.d_inuse', ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu',
       ' emiss.seq_no', ' emiss.pwr_ds_max', ' emiss.design_emi',
       ' carrier_fr.freq_carr', ' channel.bandwidth', ' channel.freq_min',
       ' channel.freq_max', 'tpaconflicts', 'percentoverlap'],
      dtype='object')


In [13]:
display(df.head(4))

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.long_nom,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,...,grp.f_biu,emiss.seq_no,emiss.pwr_ds_max,emiss.design_emi,carrier_fr.freq_carr,channel.bandwidth,channel.freq_min,channel.freq_max,tpaconflicts,percentoverlap
0,119545208,NaN,KOR,NaN,KOREASAT 3-1,NaN,9.1/IA,13.11.2019,50,1.0,...,NaN,1.0,-21.2,20M0G7D--,2057.0,20000000.0,2047.0,2067.0,2:5:,100:100
1,119545208,NaN,KOR,NaN,KOREASAT 3-1,NaN,9.1/IA,13.11.2019,50,1.0,...,NaN,3.0,-56.0,90M0G7D--,2245.0,90000000.0,2200.0,2290.0,4:11:,100:100
2,119545208,NaN,KOR,NaN,KOREASAT 3-1,NaN,9.1/IA,13.11.2019,50,1.0,...,NaN,1.0,-21.2,20M0G7D--,2067.5,20000000.0,2057.5,2077.5,5:,100
3,119545208,NaN,KOR,NaN,KOREASAT 3-1,NaN,9.1/IA,13.11.2019,50,3.0,...,NaN,1.0,-21.2,20M0G7D--,2067.5,20000000.0,2057.5,2077.5,5:,100


In [14]:
# create the extra columns
expanded = conflict_expander(df, tpadf)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
['TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R', 'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R', 'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R', 'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R', 'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E', 'TPA1.401.89525-401.90475_E', 'TPA1.2202.4-2203.4_E']


In [15]:
# save
filenameout = './databases/expanded_combined_tables_conflicts.csv'
expanded.to_csv(filenameout, index=False)
print('file saved to', filenameout)

file saved to ./databases/expanded_combined_tables_conflicts.csv


## Step 1.5 filter database for desired names

In [16]:
expanded, rejected = sat_names_isolator(expanded, os.path.join('.','satellitenames'))

filenameout = './databases/expanded_combined_tables_conflicts.csv'
expanded.to_csv(filenameout, index=False)
print('file saved to', filenameout)

filenamerej = './databases/expanded_combined_tables_conflicts_rejected.csv'
rejected.to_csv(filenamerej, index=False)
print('file saved to', filenamerej)

file saved to ./databases/expanded_combined_tables_conflicts.csv
file saved to ./databases/expanded_combined_tables_conflicts_rejected.csv


## Step 2 
separate the conflicts

In [17]:
outfolder = 'output_tables'
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
conflict_tables_separator(expanded, tpadf, outfolder)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
['TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R', 'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R', 'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R', 'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R', 'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E', 'TPA1.401.89525-401.90475_E', 'TPA1.2202.4-2203.4_E']
table saved to output_tables/TPA1.401.95525-401.96475_R_E.csv
table saved to output_tables/TPA1.401.95525-401.96475_R_R.csv
table saved to output_tables/TPA1.401.95525-401.96475_R_E_worstcase.csv
table saved to output_tables/TPA1.401.95525-401.96475_R_R_worstcase.csv
table saved to output_tables/TPA1.401.9501-401.9699_R_E.csv
table saved to output_tables/TPA1.401.9501-401.9699_R_R.csv
table saved to output_tables/TPA1.401.9501-401.9699_R_E_worstcase.csv
table saved to output_tables/TPA1.401.9501-401.9699_R_R_worstca